In [58]:
import pandas as pd
from IPython.display import display

In [59]:
file_path = "Analytics Position Case Study.xlsx"
game_df = pd.read_excel(file_path, sheet_name=1, skiprows=3)
deposit_df = pd.read_excel(file_path, sheet_name=2, skiprows=3)
withdraw_df = pd.read_excel(file_path, sheet_name=3, skiprows=3)

In [60]:
for df in [game_df, deposit_df, withdraw_df]:
    df.columns = df.columns.str.strip()

# 3. Parse Dates
for df in [game_df, deposit_df, withdraw_df]:
    df['Datetime'] = pd.to_datetime(df['Datetime'], errors='coerce')

# 4. Slot Filter Function
def get_slot_filter(df, slot, target_date):
    df = df.copy()
    df['Datetime'] = pd.to_datetime(df['Datetime'], errors='coerce')
    if slot == 'S1':
        start_time = pd.Timestamp(f"{target_date} 00:00:00")
        end_time = pd.Timestamp(f"{target_date} 11:59:59")
    else:
        start_time = pd.Timestamp(f"{target_date} 12:00:00")
        end_time = pd.Timestamp(f"{target_date} 23:59:59")
    return df[(df['Datetime'] >= start_time) & (df['Datetime'] <= end_time)]


In [61]:
# 5. Loyalty Points Calculation Function
def calculate_loyalty(game_slot, deposit_slot, withdraw_slot):
    # Aggregation
    game_agg = game_slot.groupby('User ID').agg({'Games Played': 'sum'}).reset_index()
    deposit_agg = deposit_slot.groupby('User Id').agg({
        'Amount': 'sum', 'Datetime': 'count'
    }).reset_index().rename(columns={'User Id': 'User ID', 'Amount': 'Deposit_Amount', 'Datetime': 'Deposit_Count'})
    withdraw_agg = withdraw_slot.groupby('User Id').agg({
        'Amount': 'sum', 'Datetime': 'count'
    }).reset_index().rename(columns={'User Id': 'User ID', 'Amount': 'Withdraw_Amount', 'Datetime': 'Withdraw_Count'})
    # Merge
    merged = pd.merge(game_agg, deposit_agg, on='User ID', how='outer')
    merged = pd.merge(merged, withdraw_agg, on='User ID', how='outer')
    merged = merged.fillna(0)
    # Loyalty Points Formula
    merged['Loyalty_Points'] = (
        0.01 * merged['Deposit_Amount'] +
        0.005 * merged['Withdraw_Amount'] +
        0.001 * (merged['Deposit_Count'] - merged['Withdraw_Count']).clip(lower=0) +
        0.2 * merged['Games Played']
    )
    return merged.sort_values(by='Loyalty_Points', ascending=False)

In [62]:
# 6. Dates and Slots to Analyze
slots_to_check = [
    ("2022-10-02", "S1"),
    ("2022-10-16", "S2"),
    ("2022-10-18", "S1"),
    ("2022-10-26", "S2"),
]

In [94]:
# 7. Run Analysis for Each Slot
for date, slot in slots_to_check:
    print(f"\n Slot: {date} {slot}")
    game_slot = get_slot_filter(game_df, slot, date)
    deposit_slot = get_slot_filter(deposit_df, slot, date)
    withdraw_slot = get_slot_filter(withdraw_df, slot, date)
    print(f" Games: {len(game_slot)} |  Deposits: {len(deposit_slot)} |  Withdrawals: {len(withdraw_slot)}")
    result = calculate_loyalty(game_slot, deposit_slot, withdraw_slot)
    if not result.empty:
        print(" Top 10 Players:")
        display(result[['User ID', 'Games Played', 'Deposit_Amount', 'Deposit_Count', 'Withdraw_Amount', 'Withdraw_Count', 'Loyalty_Points']].head(10))
    else:
        print("No data for this slot.")

# 8. (Optional) Save results for report
# result.to_csv("loyalty_points_results.csv", index=False)


 Slot: 2022-10-02 S1
 Games: 0 |  Deposits: 0 |  Withdrawals: 0
No data for this slot.

 Slot: 2022-10-16 S2
 Games: 5636 |  Deposits: 271 |  Withdrawals: 53
 Top 10 Players:


,User ID,Games Played,Deposit_Amount,Deposit_Count,Withdraw_Amount,Withdraw_Count,Loyalty_Points
365,634,0.0,0.0,0.0,298311.0,1.0,1491.555
122,212,0.0,99999.0,1.0,0.0,0.0,999.991
51,99,0.0,98000.0,2.0,0.0,0.0,980.002
15,28,0.0,90000.0,4.0,0.0,0.0,900.004
328,566,1.0,88000.0,3.0,0.0,0.0,880.203
414,714,0.0,61000.0,1.0,0.0,0.0,610.001
457,786,0.0,0.0,0.0,120000.0,1.0,600.000
260,455,10.0,40000.0,2.0,0.0,0.0,402.002
99,175,0.0,0.0,0.0,80000.0,1.0,400.000
38,82,0.0,0.0,0.0,75110.0,4.0,375.550



 Slot: 2022-10-18 S1
 Games: 5813 |  Deposits: 267 |  Withdrawals: 50
 Top 10 Players:


,User ID,Games Played,Deposit_Amount,Deposit_Count,Withdraw_Amount,Withdraw_Count,Loyalty_Points
391,634,0.0,0.0,0.0,544620.0,2.0,2723.100
122,208,7.0,170000.0,1.0,0.0,0.0,1701.401
419,673,4.0,90000.0,1.0,0.0,0.0,900.801
92,162,0.0,12000.0,1.0,130000.0,1.0,770.000
147,245,0.0,0.0,0.0,150000.0,1.0,750.000
213,352,4.0,50275.0,5.0,0.0,0.0,503.555
227,369,1.0,50000.0,1.0,0.0,0.0,500.201
352,569,0.0,45000.0,1.0,0.0,0.0,450.001
10,16,40.0,25000.0,4.0,33949.0,2.0,427.747
537,856,108.0,40000.0,1.0,0.0,0.0,421.601



 Slot: 2022-10-26 S2
 Games: 5849 |  Deposits: 254 |  Withdrawals: 49
 Top 10 Players:


,User ID,Games Played,Deposit_Amount,Deposit_Count,Withdraw_Amount,Withdraw_Count,Loyalty_Points
447,714,0.0,200000.0,1.0,0.0,0.0,2000.001
227,369,1.0,50000.0,1.0,200343.0,1.0,1501.915
393,634,1.0,10000.0,1.0,227362.0,1.0,1237.010
329,538,2.0,120000.0,3.0,0.0,0.0,1200.403
0,2,0.0,90000.0,2.0,0.0,0.0,900.002
175,294,0.0,16000.0,1.0,123889.0,1.0,779.445
365,587,13.0,10000.0,1.0,135000.0,1.0,777.600
110,182,44.0,65000.0,1.0,0.0,0.0,658.801
142,238,9.0,54800.0,3.0,0.0,0.0,549.803
47,80,0.0,2750.0,1.0,100000.0,1.0,527.500


In [64]:
  print(game_df[game_df['Datetime'].dt.date == pd.to_datetime("2022-10-02").date()])
  print(deposit_df[deposit_df['Datetime'].dt.date == pd.to_datetime("2022-10-02").date()])
  print(withdraw_df[withdraw_df['Datetime'].dt.date == pd.to_datetime("2022-10-02").date()])

Empty DataFrame
Columns: [User ID, Games Played, Datetime]
Index: []
Empty DataFrame
Columns: [User Id, Datetime, Amount]
Index: []
Empty DataFrame
Columns: [User Id, Datetime, Amount]
Index: []


In [65]:
for date, slot in slots_to_check:
    ...
    result = calculate_loyalty(game_slot, deposit_slot, withdraw_slot)
    if not result.empty:
        result.to_csv(f"loyalty_points_{date}_{slot}.csv", index=False)

In [66]:
print(sorted(game_df['Datetime'].dt.date.unique()))


[datetime.date(2022, 1, 10), datetime.date(2022, 2, 10), datetime.date(2022, 3, 10), datetime.date(2022, 4, 10), datetime.date(2022, 5, 10), datetime.date(2022, 6, 10), datetime.date(2022, 7, 10), datetime.date(2022, 8, 10), datetime.date(2022, 9, 10), datetime.date(2022, 10, 10), datetime.date(2022, 10, 13), datetime.date(2022, 10, 14), datetime.date(2022, 10, 15), datetime.date(2022, 10, 16), datetime.date(2022, 10, 17), datetime.date(2022, 10, 18), datetime.date(2022, 10, 19), datetime.date(2022, 10, 20), datetime.date(2022, 10, 21), datetime.date(2022, 10, 22), datetime.date(2022, 10, 23), datetime.date(2022, 10, 24), datetime.date(2022, 10, 25), datetime.date(2022, 10, 26), datetime.date(2022, 10, 27), datetime.date(2022, 10, 28), datetime.date(2022, 10, 29), datetime.date(2022, 10, 30), datetime.date(2022, 10, 31), datetime.date(2022, 11, 10), datetime.date(2022, 12, 10)]


In [90]:
oct_start = pd.Timestamp("2022-10-01")
oct_end = pd.Timestamp("2022-10-31 23:59:59")

game_oct = game_df[(game_df['Datetime'] >= oct_start) & (game_df['Datetime'] <= oct_end)]
deposit_oct = deposit_df[(deposit_df['Datetime'] >= oct_start) & (deposit_df['Datetime'] <= oct_end)]
withdraw_oct = withdraw_df[(withdraw_df['Datetime'] >= oct_start) & (withdraw_df['Datetime'] <= oct_end)]

# Step 2: Calculate loyalty points
oct_loyalty = calculate_loyalty(game_oct, deposit_oct, withdraw_oct)

# Step 3: Rank players: loyalty descending, then games played descending
oct_loyalty_sorted = oct_loyalty.sort_values(by=['Loyalty_Points', 'Games Played'], ascending=[False, False])

# Step 4: Display Top 10
print(" Top 10 Players in October:")
display(oct_loyalty_sorted.head(10))


 Top 10 Players in October:


,User ID,Games Played,Deposit_Amount,Deposit_Count,Withdraw_Amount,Withdraw_Count,Loyalty_Points
632,634,22.0,270000.0,5.0,11683352.0,50.0,61121.160
712,714,4.0,1479000.0,24.0,0.0,0.0,14790.824
210,212,0.0,1234986.0,18.0,319468.0,1.0,13947.217
670,672,8.0,1298700.0,25.0,50000.0,1.0,13238.624
97,99,4.0,817400.0,30.0,859025.0,7.0,12469.948
564,566,106.0,1209000.0,33.0,2030.0,1.0,12121.382
367,369,22.0,450000.0,9.0,1326542.0,8.0,11137.111
738,740,2.0,1058600.0,51.0,84088.0,5.0,11006.886
30,30,9.0,1004000.0,37.0,152145.0,1.0,10802.561
363,365,2368.0,279000.0,7.0,1425235.0,24.0,10389.775


In [91]:
avg_deposit = deposit_oct['Amount'].mean()
print(f"💰 Average Deposit Amount (October): ₹{avg_deposit:.2f}")


💰 Average Deposit Amount (October): ₹5604.51


In [92]:
user_deposit_totals = deposit_oct.groupby('User Id')['Amount'].sum()
avg_deposit_per_user = user_deposit_totals.mean()
print(f"💰 Average Deposit Amount per User (October): ₹{avg_deposit_per_user:.2f}")


💰 Average Deposit Amount per User (October): ₹72533.98


In [93]:
user_game_totals = game_oct.groupby('User ID')['Games Played'].sum()
avg_games_per_user = user_game_totals.mean()
print(f"🎮 Average Games Played per User (October): {avg_games_per_user:.2f}")


🎮 Average Games Played per User (October): 235.41


In [95]:
top_50 = oct_loyalty_sorted.head(50).copy()


In [97]:
top_50['Loyalty_Norm'] = top_50['Loyalty_Points'] / top_50['Loyalty_Points'].max()
top_50['Games_Norm'] = top_50['Games Played'] / top_50['Games Played'].max()

In [98]:
top_50['Final_Score'] = 0.7 * top_50['Loyalty_Norm'] + 0.3 * top_50['Games_Norm']

In [99]:
top_50['Final_Score_Norm'] = top_50['Final_Score'] / top_50['Final_Score'].sum()

In [100]:
total_bonus = 50000
top_50['Bonus_Amount'] = top_50['Final_Score_Norm'] * total_bonus

In [101]:
top_50_bonus = top_50.sort_values(by='Bonus_Amount', ascending=False)
display(top_50_bonus[['User ID', 'Loyalty_Points', 'Games Played', 'Bonus_Amount']].round(2))

,User ID,Loyalty_Points,Games Played,Bonus_Amount
632,634,61121.16,22.0,5460.06
854,856,4261.31,3869.0,2714.08
363,365,10389.78,2368.0,2354.60
986,989,6225.60,2303.0,1944.29
989,992,9503.04,1601.0,1812.81
180,182,5904.39,1572.0,1474.62
419,421,9416.35,989.0,1435.84
712,714,14790.82,4.0,1320.49
538,540,4389.04,1481.0,1284.68
210,212,13947.22,0.0,1242.90


In [ ]:
game_agg = game_slot.groupby('User ID')['Games Played'].sum()
deposit_agg = deposit_slot.groupby('User Id').agg({
    'Amount': 'sum',
    'Datetime': 'count'
})
withdraw_agg = withdraw_slot.groupby('User Id').agg({
    'Amount': 'sum',
    'Datetime': 'count'
})
